In [ ]:
!pip install MetaTrader5

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from datetime import datetime, timezone
import time
import MetaTrader5 as mt5

In [ ]:
class Indicators:
    def __init__(self, symbols, timeframes, numberofdays):
        self.symbols = symbols
        self.timeframes = timeframes
        self.numberofdays = numberofdays
        self.raw_data = {}

        # Fetch and process data for all pairs and timeframes
        self.get_data()

    def get_data(self):
        for symbol in self.symbols:
            date_from = datetime.now() - timedelta(days=self.numberofdays)
            date_to = datetime.now()

            # Fetch data for the current symbol and timeframe
            df = pd.DataFrame(mt5.copy_rates_range(symbol, self.timeframes, date_from, date_to))
            df['time'] = pd.to_datetime(df['time'], unit='s')  # Handle errors by coercing to NaT
            df.drop(columns=['tick_volume', 'real_volume'], inplace=True)
            df.set_index('time', inplace=True)

            # Resample data to 15min timeframes
            df_resampled = df.resample("15min").last()
            self.raw_data[key] = df_resampled.dropna()

        # Perform strategy for all pairs and timeframes
        self.sma_strat()

    def update_data(self):
        self.get_data()

    def sma_strat(self):
        for key, df in self.raw_data.items():
            df["SMA_10"] = df["close"].rolling(10).mean()
            df["SMA_20"] = df["close"].rolling(20).mean()
            df["SMA_50"] = df["close"].rolling(50).mean()
            df["SMA_100"] = df["close"].rolling(100).mean()
            df["SMA_200"] = df["close"].rolling(200).mean()
            
            condition_1 = (df.SMA_10 > df.SMA_20) & (df.SMA_20 > df.SMA_50) & (df.SMA_50 > df.SMA_100) & (df.SMA_100 > df.SMA_200)
            condition_2 = (df.SMA_10 < df.SMA_20) & (df.SMA_20 < df.SMA_50) & (df.SMA_50 < df.SMA_100) & (df.SMA_100 < df.SMA_200)
            
            df["position_sma"] = np.select([condition_1, condition_2], [1, -1], default=0)

    def get_signal(self, symbol, timeframe):
        df = self.raw_data.get(key, None)

        if df is not None:
            signal_value = df['position_sma'].iloc[-1]

            if signal_value == 1:
                return "buy"
            elif signal_value == -1:
                return "sell"
            else:
                return "neutral"
        else:
            return "Data not available for the specified pair and timeframe"